<a href="https://colab.research.google.com/github/3rickDJ/decision-tree/blob/main/ID3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd #for manipulating the csv data
import numpy as np #for mathematical calculation

In [20]:
import logging

In [21]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0

    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset

    return total_entr

In [22]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0

    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [23]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0

    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value

    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting

In [24]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None

    for feature in feature_list:  #for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature

    return max_info_feature

In [25]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node

    for feature_value, count in feature_value_count_dict.items():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value

        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of (feature_value = count) of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #as feature_value is not a pure class, it should be expanded further,
                                      #so the branch is marking with ?

    return tree, train_data

In [26]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None

        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]

        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with upda

In [27]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

In [28]:
train_data_m = pd.read_csv("PlayTennis.csv") #importing the dataset from the disk

train_data_m.head() #viewing some row of the dataset

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [35]:
print(train_data_m.dtypes)
train_data_m['Play Tennis']

Outlook        object
Temperature    object
Humidity       object
Wind           object
Play Tennis    object
dtype: object


0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Play Tennis, dtype: object

In [36]:
tree = id3(train_data_m, 'Play Tennis')

In [37]:
print(tree)

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}


In [46]:
train_data_cancer = pd.read_csv('entrenamiento.csv')
train_data_cancer.sample(5).columns[-1:]

Index(['ca_cervix'], dtype='object')

In [47]:
tree_cancer = id3(train_data_cancer, 'ca_cervix')

In [53]:
print(tree_cancer)

{'motivation_willingness': {8: 1, 3: {'behavior_eating': {15: 1, 12: 1, 10: 0, 13: 0}}, 5: 1, 13: {'behavior_personalHygine': {4: 1, 13: 0, 11: 0, 9: 0, 15: 0, 14: 0}}, 15: {'behavior_sexualRisk': {9: 1, 10: 0}}, 4: 1, 11: {'intention_commitment': {10: 1, 15: 0, 13: 0, 14: 0}}, 10: 0, 14: 0, 12: 0, 7: 0, 9: 0}}


In [54]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)


In [55]:
import json
jason = json.dumps(tree_cancer, indent=4, cls=NpEncoder)
print(jason)

{
    "motivation_willingness": {
        "8": 1,
        "3": {
            "behavior_eating": {
                "15": 1,
                "12": 1,
                "10": 0,
                "13": 0
            }
        },
        "5": 1,
        "13": {
            "behavior_personalHygine": {
                "4": 1,
                "13": 0,
                "11": 0,
                "9": 0,
                "15": 0,
                "14": 0
            }
        },
        "15": {
            "behavior_sexualRisk": {
                "9": 1,
                "10": 0
            }
        },
        "4": 1,
        "11": {
            "intention_commitment": {
                "10": 1,
                "15": 0,
                "13": 0,
                "14": 0
            }
        },
        "10": 0,
        "14": 0,
        "12": 0,
        "7": 0,
        "9": 0
    }
}


In [57]:
with open('ID3_tree.txt', 'w') as f:
  f.write(jason)

In [58]:
data = pd.read_csv('train.csv')
data.head()

,behavior_sexualRisk,behavior_eating,behavior_personalHygine,intention_aggregation,intention_commitment,attitude_consistency,attitude_spontaneity,norm_significantPerson,norm_fulfillment,perception_vulnerability,perception_severity,motivation_strength,motivation_willingness,socialSupport_emotionality,socialSupport_appreciation,socialSupport_instrumental,empowerment_knowledge,empowerment_abilities,empowerment_desires,ca_cervix
0,10,15,15,10,15,9,10,5,11,15,10,15,15,15,10,15,15,15,15,0
1,10,11,14,10,15,10,10,5,15,14,10,15,9,9,4,3,14,11,15,0
2,10,15,14,10,11,10,8,5,11,15,10,15,15,15,10,15,15,15,15,0
3,10,14,11,10,15,9,10,5,15,15,10,15,13,6,6,12,15,11,14,0
4,10,15,15,6,11,7,6,5,11,13,10,15,15,11,10,15,11,11,15,0


In [62]:
tree = id3(data, 'ca_cervix')
jason = json.dumps(tree, indent=4, cls=NpEncoder)

In [63]:
with open('ID3_tree_own_set.txt', 'w') as f:
  f.write(json.dumps(tree, indent=4, cls=NpEncoder))

In [64]:
print(jason)

{
    "norm_fulfillment": {
        "11": 0,
        "15": 0,
        "14": 0,
        "13": 0,
        "3": {
            "empowerment_knowledge": {
                "12": 0,
                "15": 0,
                "6": 0,
                "8": 1,
                "3": 1,
                "13": 1,
                "7": 1
            }
        },
        "5": {
            "behavior_eating": {
                "13": 0,
                "15": 0,
                "10": 0,
                "14": 0,
                "11": 1,
                "12": 1
            }
        },
        "4": {
            "behavior_eating": {
                "12": 0,
                "11": 0,
                "15": 1
            }
        },
        "7": {
            "behavior_eating": {
                "15": 0,
                "12": 1
            }
        },
        "10": 1,
        "8": 1,
        "6": 1,
        "12": 1
    }
}
